# Last.fm

In [8]:
import requests
import json
import time
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

### Function used for data scripting

In [9]:
#shared secret:ae1ca19ada9ac3d11d2a2127b339545b
def lastfm_get(payload):

    headers = {'user-agent': 'rzhan97'}
    url = 'http://ws.audioscrobbler.com/2.0/'

    payload['api_key'] = '5e11e4bca8ad54566e466513df709822'
    payload['format'] = 'json'

    response = requests.get(url, headers=headers, params=payload)
    return response

#This function is only help for me to understand json file
def jprint(obj):
    text = json.dumps(obj, sort_keys=True, indent=4)
    print(text)

In [10]:
def lookup_userinfo(user):
    response = lastfm_get({
    'method': 'user.getInfo',
    'user' : user
    })

    if response.status_code != 200:
        return None

    name = response.json()['user']['realname']
    country = response.json()['user']['country']
    playcount = response.json()['user']['playcount']


    # rate limiting
    if not getattr(response, 'from_cache', False):
        time.sleep(0.25)
    return name,country,playcount

In [11]:
#Build a funtion to get user's top track, I also wrote a function about user's loved song and recent songs
#However, recent songs cannot represent user like it or not, and the loved song 
def lookup_topsong(user):
    response = lastfm_get({
    'method': 'user.getTopTracks',
    'user' : user
    })

    if response.status_code != 200:
        return None

    song = [t['name'] for t in response.json()['toptracks']['track']]
    playcount = [t['playcount'] for t in response.json()['toptracks']['track']]
    artist = [t['artist']['name'] for t in response.json()['toptracks']['track']]


    # rate limiting
    if not getattr(response, 'from_cache', False):
        time.sleep(0.25)
    return song,playcount,artist

In [12]:
#Build a funtion to get user's library
def lookup_songinfo(track,artist):
    response = lastfm_get({
    'method': 'track.getInfo',
    'track' : track,
    'artist':artist
    })

    if response.status_code != 200:
        return None

    duration = response.json()['track']['duration']
    listeners = response.json()['track']['listeners']
    playcount = response.json()['track']['playcount']
    album = response.json()['track']['album']['title'] 
    tags = [t['name'] for t in response.json()['track']['toptags']['tag'] ]
    try:
        published_date = response.json()['track']['wiki']['published']
    except:
        published_date = None

    # rate limiting
    if not getattr(response, 'from_cache', False):
        time.sleep(0.25)
    return duration,listeners,playcount,album,tags,published_date

### Get user'name

In [15]:
#Get my neighbors' neighbors' neighbors, which is 3rd connection with me, to reduce some bias
from selenium import webdriver
from selenium.webdriver.common.action_chains import ActionChains
import time
def get_users(username):
    path = "/Users/zoe/Desktop/chromedriver"
    driver = webdriver.Chrome(path)
    driver.get("https://www.last.fm/user/"+username+"/neighbours")
    user = []
    user_names = driver.find_elements_by_class_name("user-list-name")
    for i in range(len(user_names)):
        time.sleep(2)
        user_link = user_names[i].find_element_by_tag_name('a').get_attribute('href')+"/neighbours"
        time.sleep(2)
        driver.get(user_link)
        user_names = driver.find_elements_by_class_name("user-list-name")
        for i in range(len(user_names)):
            time.sleep(2)
            user_link = user_names[i].find_element_by_tag_name('a').get_attribute('href')+"/neighbours"
            time.sleep(2)
            driver.get(user_link)
            user_names = driver.find_elements_by_class_name("user-list-name")
            for i in range(len(user_names)):
                user.append(user_names[i].text)
            driver.back()
            time.sleep(2)
            user_names = driver.find_elements_by_class_name("user-list-name")
        driver.back()
        time.sleep(2)
        user_names = driver.find_elements_by_class_name("user-list-name")
    user = list(dict.fromkeys(user))
    driver.quit()
    return user

In [17]:
rzhan97_list = get_users("rzhan97")

IndexError: list index out of range

In [52]:
path = "/Users/zoe/Desktop/chromedriver"
driver = webdriver.Chrome(path)
driver.get("https://www.last.fm/user/rzhan97/neighbours")
user = []
user_names = driver.find_elements_by_class_name("user-list-name")
for i in range(len(user_names)):
    time.sleep(2)
    user_link = user_names[i].find_element_by_tag_name('a').get_attribute('href')+"/neighbours"
    time.sleep(2)
    driver.get(user_link)
    user_names = driver.find_elements_by_class_name("user-list-name")
    for i in range(len(user_names)):
        time.sleep(2)
        user_link = user_names[i].find_element_by_tag_name('a').get_attribute('href')+"/neighbours"
        time.sleep(2)
        driver.get(user_link)
        user_names = driver.find_elements_by_class_name("user-list-name")
        for i in range(len(user_names)):
            user.append(user_names[i].text)
        driver.back()
        time.sleep(2)
        user_names = driver.find_elements_by_class_name("user-list-name")
    driver.back()
    time.sleep(2)
    user_names = driver.find_elements_by_class_name("user-list-name")
user = list(dict.fromkeys(user))
driver.quit()

KeyboardInterrupt: 

In [53]:
#See how many users we get
print(len(user))


299


In [28]:
import pickle
with open("user.txt", "wb") as fp:   #Pickling
    pickle.dump(user, fp)

In [29]:
import pickle
new_user = []
with (open("user.txt", "rb")) as openfile:
    while True:
        try:
            user.append(pickle.load(openfile))
        except EOFError:
            break

In [54]:
rzhan97_small_list = user
print(len(rzhan97_small_list))

299


In [30]:
from random import sample
rzhan97_small_list= sample(user,10000)

ValueError: Sample larger than population or is negative

In [66]:
print(rzhan97_small_list)

['achee_', 'tetepopfm', 'deathdivisi0n', 'redliptwo', 'Redlipno', 'Ellie--04', 'Asclo', 'collinsen6', 'aviesaptarini', 'haseullie', 'heiyooo', 'Lemoonjpeg', 'njmiso', 'garoulobito', 'PocariSweat', 'ActScene1', 'venticachaceiro', 'Jyul', 'Sw4gapino', 'yoonjenny', 'kimdh90903', 'lydiabradleym', 'NotMonako', 'samantha033095', 'Judd46', 'Chuu7', 'NYUC0RE', 'HugoisanOmega', 'eugeniamedieval', 'grwlsb', 'DylanTay', 'yojees', 'takadable', 'hqnniq', 'wiarae', 'Stantiara', 'sunwoosgaymerbf', 'taiyankee', 'vscogirl', 'cooheart', 'ineshr', 'Dcmantuan', 'aleperez_144', 'eojjeona', 'moiraac', 'cisopa8415', 'allioe', 'rexthegodz', 'whe1998', 'lucasdeusrey', 'sanskelisia', 'minarska_', 'tuvada', 'm8nape', 'pyotrmarche', 'Nikki_ink', 'violenceultra2', 'liamerkowitz', 'maryyy6783', 'kantojoey', 'chabmeister', 'adazhng', 'Cherrybaby4', 'emmaluvs2dance', 'kjswift18', 'mralecgasin', 'Claraqw', 'iona_mckie', 'sophlua', 'Eburge70', 'Leahhhhhh18', 'kamryngresko', 'beckbennett', 'haleena_s', 'HanahAlcantara',

In [88]:
#convert them into dataframe, also get the artist with the song, in case there are duplicate song name
song = []
artist = []
count = []
user = []
for i in range(len(rzhan97_small_list)):
    data = lookup_topsong(rzhan97_small_list[i])
    print(rzhan97_small_list[i])
    song.append(data[0])
    count.append(data[1])
    artist.append(data[2])
    user.append(rzhan97_small_list[i] * len(count))


usersong_df = pd.DataFrame(zip(user,artist,song,count), columns =['user','artist','songs','count'])


achee_
tetepopfm
deathdivisi0n
redliptwo
Redlipno
Ellie--04
Asclo
collinsen6
aviesaptarini
haseullie
heiyooo
Lemoonjpeg
njmiso
garoulobito
PocariSweat
ActScene1
venticachaceiro
Jyul
Sw4gapino
yoonjenny
kimdh90903
lydiabradleym
NotMonako
samantha033095
Judd46
Chuu7
NYUC0RE
HugoisanOmega
eugeniamedieval
grwlsb
DylanTay
yojees
takadable
hqnniq
wiarae
Stantiara
sunwoosgaymerbf
taiyankee
vscogirl
cooheart
ineshr
Dcmantuan
aleperez_144
eojjeona
moiraac
cisopa8415
allioe
rexthegodz
whe1998
lucasdeusrey
sanskelisia
minarska_
tuvada
m8nape
pyotrmarche
Nikki_ink
violenceultra2
liamerkowitz
maryyy6783
kantojoey
chabmeister
adazhng
Cherrybaby4
emmaluvs2dance
kjswift18
mralecgasin
Claraqw
iona_mckie
sophlua
Eburge70
Leahhhhhh18
kamryngresko
beckbennett
haleena_s
HanahAlcantara
rhysmjames
chonkpoop
colormebIue
SoczekYSS
Brxtt18
iriswei36
Isabella_p22
aslan-kn
mon3ypowerglory
averywild
lydiahmm
georgialonergan
nefelibraoudaki
elina_fay
jacobshapiro
toomuchtoask_
bellegmsss
shitecraic
Sophiemelissa
sw

In [90]:
def unnesting(df, explode):
    idx = df.index.repeat(df[explode[0]].str.len())
    df1 = pd.concat([
        pd.DataFrame({x: np.concatenate(df[x].values)}) for x in explode], axis=1)
    df1.index = idx

    return df1.join(df.drop(explode, 1), how='left')

        
usersong_df = unnesting(usersong_df,['songs','count','artist'])
usersong_df.head()

,songs,count,artist,user
0,Cure For Me,251,Aurora,achee_
0,Cabaret,247,Azizi Gibson,achee_
0,You Are the Right One,179,Sports,achee_
0,rap music,149,Lil Darkie,achee_
0,Wet The Bed,139,Sik-K,achee_


In [91]:
song_df = pd.DataFrame()
usersong_rating = usersong_df
song_df = usersong_df
song_df.head()

,songs,count,artist,user
0,Cure For Me,251,Aurora,achee_
0,Cabaret,247,Azizi Gibson,achee_
0,You Are the Right One,179,Sports,achee_
0,rap music,149,Lil Darkie,achee_
0,Wet The Bed,139,Sik-K,achee_


In [106]:
usersong_rating = usersong_rating.
usersong_rating.head(10)

,songs,count,artist,user,song
0,Cure For Me,251,Aurora,achee_,Aurora-Cure For Me
0,Cabaret,247,Azizi Gibson,achee_,Azizi Gibson-Cabaret
0,You Are the Right One,179,Sports,achee_,Sports-You Are the Right One
0,rap music,149,Lil Darkie,achee_,Lil Darkie-rap music
0,Wet The Bed,139,Sik-K,achee_,Sik-K-Wet The Bed
0,JUGO DE MELÓN,131,cardellino,achee_,cardellino-JUGO DE MELÓN
0,Anillos,128,Noah Pino Palo,achee_,Noah Pino Palo-Anillos
0,Company,104,Lucid State,achee_,Lucid State-Company
0,THERAPY,103,Lil Darkie,achee_,Lil Darkie-THERAPY
0,Pick Up Your Phone,99,Hojean,achee_,Hojean-Pick Up Your Phone


In [107]:
usersong_rating['songs'] = usersong_rating["artist"] + "-" + usersong_rating["song"]
usersong_rating = usersong_rating.drop(['artist'], axis=1)
usersong_rating = usersong_rating.drop(['song'], axis=1)
usersong_rating.head(10)

,songs,count,user
0,Aurora-Aurora-Cure For Me,251,achee_
0,Azizi Gibson-Azizi Gibson-Cabaret,247,achee_
0,Sports-Sports-You Are the Right One,179,achee_
0,Lil Darkie-Lil Darkie-rap music,149,achee_
0,Sik-K-Sik-K-Wet The Bed,139,achee_
0,cardellino-cardellino-JUGO DE MELÓN,131,achee_
0,Noah Pino Palo-Noah Pino Palo-Anillos,128,achee_
0,Lucid State-Lucid State-Company,104,achee_
0,Lil Darkie-Lil Darkie-THERAPY,103,achee_
0,Hojean-Hojean-Pick Up Your Phone,99,achee_


In [109]:
usersong_matrix_df = usersong_rating.pivot(index='user', columns='songs', values='count').fillna(0)
usersong_matrix_df.head(50)

songs,$NOT-$NOT-4,$NOT-$NOT-FALL IN LOVE,$NOT-$NOT-GOSHA,$NOT-$NOT-Like Me (feat. iann dior),$NOT-$NOT-Moon & Stars (feat. Maggie Lindemann),$NOT-$NOT-ROADRUNNER +,$NOT-$NOT-VINTAGE DIOR,$NOT-$NOT-Whipski (feat. Lil Skies & Internet Money),$atori Zoom-$atori Zoom-barracuda,"$uicideboy$-$uicideboy$-...And To Those I Love, Thanks For Sticking Around",...,새소년 Se So Neon-새소년 Se So Neon-A Long Dream 긴 꿈,새소년 Se So Neon-새소년 Se So Neon-NAN CHUN,선미-선미-주인공,신승훈-신승훈-I Believe,"어때요 노래 참 쉽죠? 밥로스-어때요 노래 참 쉽죠? 밥로스-한숨 (어때요, 노래 참 쉽죠? 밥로스) - Live Version",워너원-워너원-에너제틱 (Energetic),유키카-유키카-Insomnia,"태양-태양-Eyes, Nose, Lips","태양-태양-눈,코,입(Eyes, Noes, Lips)",티아라-티아라-Sugar Free
user,,,,,,,,,,,,,,,,,,,,,
AaronohearAaronohearAaronohearAaronohearAaronohearAaronohearAaronohearAaronohearAaronohearAaronohearAaronohearAaronohearAaronohearAaronohearAaronohearAaronohearAaronohearAaronohearAaronohearAaronohearAaronohearAaronohearAaronohearAaronohearAaronohearAaronohearAaronohearAaronohearAaronohearAaronohearAaronohearAaronohearAaronohearAaronohearAaronohearAaronohearAaronohearAaronohearAaronohearAaronohearAaronohearAaronohearAaronohearAaronohearAaronohearAaronohearAaronohearAaronohearAaronohearAaronohearAaronohearAaronohearAaronohearAaronohearAaronohearAaronohearAaronohearAaronohearAaronohearAaronohearAaronohearAaronohearAaronohearAaronohearAaronohearAaronohearAaronohearAaronohearAaronohearAaronohearAaronohearAaronohearAaronohearAaronohearAaronohearAaronohearAaronohearAaronohearAaronohearAaronohearAaronohearAaronohearAaronohearAaronohearAaronohearAaronohearAaronohearAaronohearAaronohearAaronohearAaronohearAaronohearAaronohearAaronohearAaronohearAaronohearAaronohearAaronohearAaronohearAaronohearAaronohearAaronohearAaronohearAaronohearAaronohearAaronohearAaronohearAaronohearAaronohearAaronohearAaronohearAaronohearAaronohearAaronohearAaronohearAaronohearAaronohearAaronohearAaronohearAaronohearAaronohearAaronohearAaronohearAaronohearAaronohearAaronohearAaronohearAaronohearAaronohearAaronohearAaronohearAaronohearAaronohearAaronohearAaronohearAaronohearAaronohearAaronohearAaronohearAaronohearAaronohearAaronohearAaronohearAaronohearAaronohearAaronohearAaronohearAaronohearAaronohearAaronohearAaronohearAaronohearAaronohearAaronohearAaronohearAaronohearAaronohearAaronohearAaronohearAaronohearAaronohearAaronohearAaronohearAaronohearAaronohearAaronohearAaronohearAaronohearAaronohearAaronohearAaronohearAaronohearAaronohearAaronohearAaronohearAaronohearAaronohearAaronohearAaronohearAaronohearAaronohearAaronohearAaronohearAaronohearAaronohearAaronohearAaronohearAaronohearAaronohearAaronohearAaronohearAaronohearAaronohearAaronohearAaronohearAaronohearAaronohearAaronohearAaronohearAaronohearAaronohearAaronohearAaronohearAaronohearAaronohearAaronohearAaronohearAaronohearAaronohearAaronohearAaronohearAaronohearAaronohearAaronohearAaronohearAaronohearAaronohearAaronohearAaronohearAaronohearAaronohearAaronohearAaronohearAaronohearAaronohearAaronohearAaronohearAaronohearAaronohearAaronohearAaronohearAaronohearAaronohearAaronohearAaronohearAaronohearAaronohearAaronohearAaronohearAaronohearAaronohearAaronohearAaronohearAaronohearAaronohearAaronohearAaronohearAaronohearAaronohearAaronohearAaronohearAaronohearAaronohearAaronohearAaronohearAaronohearAaronohearAaronohearAaronohearAaronohearAaronohearAaronohearAaronohearAaronohearAaronohearAaronohearAaronohearAaronohearAaronohearAaronohearAaronohearAaronohearAaronohearAaronohearAaronohearAaronohearAaronohearAaronohearAaronohearAaronohearAaronohearAaronohearAaronohearAaronohearAaronohearAaronohearAaronohearAaronohearAaronohearAaronohearAaronohearAaronohearAaronohearAaronohear,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
AbdullahQawwasAbdullahQawwasAbdullahQawwasAbdullahQawwasAbdullahQawwasAbdullahQawwasAbdullahQawwasAbdullahQawwasAbdullahQawwasAbdullahQawwasAbdullahQawwasAbdullahQawwasAbdullahQawwasAbdullahQawwasAbdullahQawwasAbdullahQawwasAbdullahQawwasAbdullahQawwasAbdullahQawwasAbdullahQawwasAbdullahQawwasAbdullahQawwasAbdullahQawwasAbdull

In [ ]:
usersong_rating_df

In [ ]:
usersong_df.head()

In [ ]:
#user info
name = []
country = []
playcount = []
for i in range(len(rzhan97_small_list)):
    data = lookup_userinfo(rzhan97_small_list[i])
    name.append(data[0])
    country.append(data[1])
    playcount.append(data[2])

usersong_df = pd.DataFrame(zip(rzhan97_small_list,name ,country,playcount), columns =['user','name','song','count'])
#usersong_df['count'] = usersong_df['count'].astype(int)

In [ ]:
song_df = usersong_df[['artist','songs']]
song_df.drop_duplicates()
song_df = song_df.apply(
      lambda x: lookup_songinfo(x['songs'],x['artist'],
                                index=['duration', 'listeners','playcount','album','tags','published_date']),
    axis=1

song_df.head()

    